In [3]:
!pip install espn_api
!pip install pygsheets
from espn_api.basketball import League
from datetime import datetime, timedelta
import pandas as pd

  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [espn_api]1/2 [espn_api]
  Using cached pygsheets-2.0.6-py3-none-any.whl.metadata (14 kB)
Using cached pygsheets-2.0.6-py3-none-any.whl (158 kB)


In [4]:
import pygsheets
def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True, index = False)
    wks_write.frozen_rows = 1
serv_path = 'keykey.json'
SPREADSHEET_ID = '1Ku6qmPGoKjZ16NUBLCdopkNI-qw4BnseQGDUQXa_oko'
sheet_name = 'My team API'
#write_to_gsheet(serv_path, SPREADSHEET_ID, sheet_name, quinn_m)

In [5]:
league = League(league_id=403382108, year=2025, espn_s2='AECe11iWbFkute%2FcY4wPm5PnP5LUWP13PANIDlEASVbSasrkLRJ15a%2F%2FtRwmnMpFI%2FSSHhyqR1X4GntAc6HshqRDwVFmzrrR52X5Bq8qcXk1ryfkfHcfqkRQA6p8%2Bf4Yw5tpjgS7j2ceNsMjz4GGzYKGo3a2%2FvTXOCszrRK%2BDA5J0QXu9jBhyaoaas83MQ%2F%2BZMQbG6yvVyVbovujmTCxmIvcZf1UPDsRa5zaYubuxQQdznBqXjVjl%2Ff3V6VGdwolqNEbEmsQFItF9ROaX0GSPozOgSsRn5LmeDXSx6fEt%2FfxjA%3D%3D', swid='{FC544995-8240-4FE8-9449-958240FFE88D}')

In [6]:
matchups = league.scoreboard()

# Loop through matchups to find starting players and make them into lists
starters = {}
for matchup in matchups:

    home_starters = []
    away_starters = []
    
    for player in matchup.home_team.roster:
        if player.lineupSlot not in ['BE', 'IR']:  
            home_starters.append(player)
    if matchup.away_team != 0:
        for player in matchup.away_team.roster:
            if player.lineupSlot not in ['BE', 'IR']:
                away_starters.append(player)

    else:
        away_starters = [0]
        

    starters[matchup.home_team] = home_starters
    starters[matchup.away_team] = away_starters


In [7]:
matchups #check that they match

[Matchup(Team(Nice Try Giddey), Team(Los Guangdong Mapaches)),
 Matchup(Team(OH AMEN STOP THAT), Team(KotaroLivesAlone)),
 Matchup(Team(Subtle Off-White Coloring), Team(Flashy like Primo)),
 Matchup(Team(Jonathaned in the Poole), Team(Yong)),
 Matchup(Team(players I'd give my meniscus to), Team(My Sweet Glorious King)),
 Matchup(Team(Anything Is Possible), Team(Don't Ask))]

In [8]:
#create the livescore dataframe and save to pickle format

today = (datetime.today() - timedelta(days=datetime.today().weekday())).replace(hour=0, minute=0, second=0, microsecond=0)
nextmon = today + timedelta(days=14)
days = []
for i in range(7):
    days.append((today + timedelta(days=i)))
matched_teams = []
for i in league.box_scores():
    home = str(i.home_team)
    away = str(i.away_team)
    matched_teams.append(home)
    matched_teams.append(away)
    
    
livescores = pd.DataFrame(index = matched_teams, columns = days)
livescores.columns = livescores.columns.astype(str)
livescores2 = livescores.copy()
livescores = livescores.to_pickle("livescores.pkl")
livescores2.reset_index(inplace=True)

write_to_gsheet(serv_path, SPREADSHEET_ID, 'Livescores', livescores2)

In [ ]:
#matchups
team_matchups = []
for i in matchups:
    team_matchups.append(i.home_team)
    team_matchups.append(i.away_team)
team_matchups
matchup_df = pd.DataFrame({'Team': team_matchups})

# Insert blank rows after every two rows
new_index = []
for i in range(len(matchup_df)):
    new_index.append(i)
    if (i + 1) % 2 == 0:  # Add a new blank row after every two rows
        new_index.append(f'blank_{i // 2}')

matchup_df = matchup_df.reindex(new_index)
write_to_gsheet(serv_path, SPREADSHEET_ID, 'Matchups', matchup_df)
team_matchups

In [ ]:
#daily loop through rosters to adjust each players average points and set injured players to 0 for the day
def final_prod(team):

    #today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    currentmon = (datetime.today() - timedelta(days=datetime.today().weekday())).replace(hour=0, minute=0, second=0, microsecond=0)
    today = currentmon
    nextmon = today + timedelta(days=7)
    days = []
    for i in range(7):
        days.append((today + timedelta(days=i)))
    def date_extracter(roster):
        date_dict = {}
        for i in roster:
            sched = list(dict(sorted(i.schedule.items(), key=lambda item: int(item[0]))).values())
            games = []
            for game in sched:
                if nextmon>game['date']> today:
                    games.append(game['date'].replace(hour=0, minute=0, second=0, microsecond=0))
            date_dict[i.name] = games
    
        return date_dict

    def matrix_maker(games, day_list):
    
        # Initialize an empty list to store the matrix
        matrix = []
        
        # Iterate through each person and create their row in the matrix
        for person, dates in games.items():
            row = []
            for date in day_list:
                # Append 1 if the person is going on that date, otherwise append 0
                if date in dates:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        #check starters
        # Convert the matrix to a DataFrame for better visualization
        game_matrix = pd.DataFrame(matrix, columns=day_list, index=games.keys())
        game_matrix['Average Points'] = [i.avg_points for i in starters[team]]
        others = list(game_matrix.columns[0:14])
        others.insert(0,'Average Points')
        return game_matrix[others]

    games = date_extracter(starters[team])
    fin = matrix_maker(games,days)
    rn = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    for i in starters[team]:
        if i.injured == True:
            fin.loc[i.name, rn] = 0
    fin.columns = fin.columns.astype(str)
    fin.reset_index(inplace=True)
    fin = fin.rename(columns = {'index': team})
    fin.columns = fin.columns.astype(str)
    

    return fin

#to sheets
for i in league.teams:
    write_to_gsheet(serv_path, SPREADSHEET_ID, str(i), final_prod(i))

